In [ ]:
import pandas as pd
f = pd.read_csv("titanic.csv")
f.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
f.Age.fillna(f.Age.median(), inplace=True)
# 여자:0 남자:1로 변환
f.Sex = f.Sex.map({'female':0, 'male':1})

# Embarked 누락값을 S로
f.Embarked.fillna('S', inplace=True)

embarked_dummies = pd.get_dummies(f.Embarked, prefix='Embarked', drop_first=True)

# 뒤에 붙여줌
titanic = pd.concat([f, embarked_dummies], axis=1)

In [ ]:
f.corr() # Pclass, Sex, Fare	 

<ipython-input-3-6765426f71c1>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  f.corr() # Pclass, Sex, Fare


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.034212,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.064910,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.339898,0.083081,0.018443,-0.549500
Sex,0.042939,-0.543351,0.131900,1.000000,0.081163,-0.114631,-0.245489,-0.182333
Age,0.034212,-0.064910,-0.339898,0.081163,1.000000,-0.233296,-0.172482,0.096688
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.233296,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.172482,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,-0.182333,0.096688,0.159651,0.216225,1.000000


In [ ]:
!pip install pycaret

In [ ]:
# pycaret classification 모듈
from pycaret.classification import setup
# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models
# 컬럼명 String으로 반환
# 데이터셋 설정
grid = setup(data=f, target="Survived",  verbose=True)
# 모델 학습
best = compare_models()
# best 모델 확인
print(best)

,Description,Value
0,Session id,3259
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 14)"
5,Transformed train set shape,"(623, 14)"
6,Transformed test set shape,"(268, 14)"
7,Numeric features,7
8,Categorical features,4
9,Rows with missing values,77.1%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8073,0.0000,0.6563,0.8159,0.7210,0.5771,0.5900,0.2890
lr,Logistic Regression,0.8025,0.8532,0.6650,0.7950,0.7193,0.5695,0.5789,0.8950
lda,Linear Discriminant Analysis,0.8024,0.8539,0.6987,0.7757,0.7302,0.5754,0.5819,0.3690
et,Extra Trees Classifier,0.7931,0.8425,0.6281,0.7953,0.6955,0.5436,0.5562,0.8730
nb,Naive Bayes,0.7494,0.8206,0.4897,0.7773,0.5943,0.4293,0.4555,0.2870
rf,Random Forest Classifier,0.7384,0.8461,0.3853,0.8483,0.5233,0.3814,0.4382,0.7290
ada,Ada Boost Classifier,0.7192,0.8124,0.3391,0.8227,0.4705,0.3297,0.3898,0.5190
dt,Decision Tree Classifier,0.7111,0.6395,0.3310,0.7874,0.4565,0.3110,0.3648,0.2870
lightgbm,Light Gradient Boosting Machine,0.7111,0.7428,0.3306,0.8019,0.4610,0.3120,0.3704,0.4310
gbc,Gradient Boosting Classifier,0.7095,0.7291,0.3268,0.7986,0.4535,0.3074,0.3650,0.7300


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=3259, solver='auto',
                tol=0.0001)


In [ ]:
X = f[['Pclass', 'Sex', 'Fare']]
y = f.Survived

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2 , random_state=42)



In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
import optuna
import numpy as np
import sklearn
import xgboost as xgb
def objective(trial):

    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
      
    }


    model = xgb.XGBClassifier(**params, random_state = 7)

    bst = model.fit(X_train, y_train,eval_set=[(X_valid,y_valid)], early_stopping_rounds=50, verbose=False)
    preds = bst.predict(X_valid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")


trial = study.best_trial


[I 2023-04-18 06:23:50,655] A new study created in memory with name: no-name-992ec9c6-01f5-4f2e-b08c-b8f5308362ca


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-04-18 06:23:50,782] Trial 0 finished with value: 0.8044692737430168 and parameters: {'max_depth': 5, 'learning_rate': 0.4587561600407217, 'n_estimators': 9600}. Best is trial 0 with value: 0.8044692737430168.
[I 2023-04-18 06:23:50,855] Trial 1 finished with value: 0.7988826815642458 and parameters: {'max_depth': 9, 'learning_rate': 0.3126782401496461, 'n_estimators': 6100}. Best is trial 0 with value: 0.8044692737430168.
[I 2023-04-18 06:23:50,942] Trial 2 finished with value: 0.8156424581005587 and parameters: {'max_depth': 9, 'learning_rate': 0.5406510875536655, 'n_estimators': 1200}. Best is trial 2 with value: 0.8156424581005587.
[I 2023-04-18 06:23:51,199] Trial 3 finished with value: 0.8156424581005587 and parameters: {'max_depth': 9, 'learning_rate': 0.5441658752297063, 'n_estimators': 5200}. Best is trial 2 with value: 0.8156424581005587.
[I 2023-04-18 06:23:51,346] Trial 4 finished with value: 0.8100558659217877 and parameters: {'max_depth': 6, 'learning_rate': 0.6379

In [ ]:
from sklearn.ensemble import  StackingClassifier

# base_models 설정
base_models = [
                 ('rc', RidgeClassifier()),
                 ('lr', LogisticRegression(base_estimator=DecisionTreeClassifier()),
                  ('lr', ExtraTreesClassifier(base_estimator=DecisionTreeClassifier())
                ]

# stacking 설정
clf = StackingClassifier(estimators=base_models, final_estimator= CatboostClassfier() )

clf.fit(X_train,y_train)

In [ ]:
# 대회에 제출할 csv 파일을 생성